### Assignment 2-Workload 2

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.ml.recommendation import *

In [2]:
spark = SparkSession \
    .builder \
    .appName("Assignment2_workload2") \
    .getOrCreate()
sc =SparkContext.getOrCreate()

### Data preparation

In [3]:
#Loading rawdata
rawdata = spark.read.option("multiline","true").json("tweets.json")
# rawdata.show(1)
usermention = rawdata.select("user_id","user_mentions")
# usermention.show(10, truncate=False)

#convert user id to fit ALS(longint cannot be accepted by ALS)
update_user_id = rawdata.select("user_id") \
                 .rdd.map(lambda x: x[0]) \
                 .distinct().collect()

new_user_id = dict()

for item in update_user_id:
    new_user_id[item] = len(new_user_id)

#convert user_mentions to fit ALS(longint cannot be accepted by ALS)
update_user_mentions =  rawdata.select("user_mentions") \
                               .where("user_mentions is not null") \
                               .rdd.flatMap(lambda x: [y[0] for y in x[0]]) \
                               .distinct().collect()

new_user_mentions = dict()

for item in update_user_mentions:
    new_user_mentions[item] = len(new_user_mentions)

#broadcast new_user_id and new_user_mentions to all nodes.
sc.broadcast(new_user_id)
sc.broadcast(new_user_mentions)

### User Recommendation

In [4]:
#convert user_id and user_mention to pair and count
def user_mention_cov(x):
    user_id = x[0]
    user_mention_list = list()
    for user_mentions in x[1]:
        user_mention_list.append(tuple([user_id,new_user_mentions[user_mentions[0]],1]))
    return user_mention_list

#archieve the RDD with user_id, user_mention and mention_count, which can be fed to ALS.
usermention_rdd = usermention.where("user_mentions is not null") \
                             .rdd.map(lambda x: (new_user_id[x[0]],x[1])) \
                             .flatMap(user_mention_cov)
usermention_rdd_count = usermention_rdd.map(lambda x: ((x[0],x[1]),x[2])) \
                                       .reduceByKey(lambda x1,x2: x1+x2) \
                                       .map(lambda x: (x[0][0],x[0][1],x[1])) \
                                       .toDF(["user_id","user_mentions","mention_count"])

#Collaborative Filtering
als =ALS(userCol="user_id", itemCol="user_mentions", ratingCol = "mention_count", coldStartStrategy="drop")
model = als.fit(usermention_rdd_count)

#get the top 5 recoomondation
userRecs = model.recommendForAllUsers(5).collect()

print("Top 5 mention user for each user are")

for row in userRecs:
    print(update_user_id[row[0]],":", end = " ")
    for b in row[1]:
        print(update_user_mentions[b[0]], end = " ")
    print()

Top 5 mention user for each user are
17375281 : 111404710 61863570 281877818 14195880 8083722 
56722736 : 24259259 346102488 1244013067790364673 17674244 3094649957 
866949673461981184 : 281877818 2374745089 61863570 15859912 186520006 
776625931234390017 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1220079661201399809 : 380648579 32828872 100248020 1656958189 6318552 
107548394 : 24259259 56488059 20457806 26574283 1891266440 
14849092 : 24259259 56488059 20457806 26574283 1891266440 
319372040 : 24259259 56488059 20457806 26574283 1891266440 
325856719 : 15115280 17674244 24259259 224329419 1643123766 
83105742 : 83440337 260371201 17674244 61521038 860654500431880192 
1241184520348590080 : 24259259 56488059 20457806 26574283 1891266440 
1478659146 : 24259259 56488059 20457806 26574283 1891266440 
4474623749 : 133081348 408793153 1267708379138535424 24259259 20457806 
116083254 : 24259259 56488059 20457806 26574283 1891266440 
3252147534 : 133081348 408793153 1267708

31001654 : 24259259 346102488 1244013067790364673 17674244 3094649957 
38649374 : 56488059 20457806 24259259 939091 1349149096909668363 
1153116974379098112 : 24259259 1643123766 3044682647 3094649957 26574283 
299856713 : 24259259 56488059 20457806 26574283 1891266440 
32543768 : 24259259 346102488 1244013067790364673 17674244 3094649957 
829039206362025984 : 15115280 17674244 24259259 224329419 1643123766 
1357541074647064576 : 20457806 56488059 24259259 207809313 471741741 
1365883489246208001 : 24259259 346102488 1244013067790364673 17674244 3094649957 
141031627 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1103694806319054851 : 15859912 41704988 17112878 794521198717706240 47678782 
842363965 : 24259259 56488059 20457806 26574283 1891266440 
47369119 : 24259259 346102488 1244013067790364673 17674244 3094649957 
262303890 : 24259259 56488059 20457806 26574283 1891266440 
52094248 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1381006648270069762 : 24

80791048 : 56488059 20457806 24259259 1349149096909668363 1858780626 
1294640158172745731 : 15115280 17674244 24259259 224329419 1643123766 
45785525 : 24259259 56488059 20457806 26574283 1891266440 
252412547 : 20457806 56488059 18831926 335060282 346102488 
16674252 : 8973062 216647845 20457806 56488059 3240396234 
318566998 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1285411922494586881 : 24259259 346102488 1244013067790364673 17674244 3094649957 
778172370481778688 : 24259259 346102488 1244013067790364673 17674244 3094649957 
20227046 : 24259259 56488059 20457806 26574283 1891266440 
1381808105735000064 : 24259259 346102488 3094649957 1244013067790364673 17674244 
824684311827738624 : 24259259 20457806 56488059 26574283 185025785 
1356059064741064720 : 24259259 56488059 20457806 26574283 1891266440 
2859771291 : 24259259 56488059 20457806 26574283 1891266440 
443688861 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1101740211770482690 : 56488059 204

807303262554701824 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1317905130432450564 : 24259259 346102488 3094649957 1244013067790364673 17674244 
259493695 : 24259259 346102488 1244013067790364673 17674244 3094649957 
2420618906 : 56488059 20457806 24259259 939091 2420618906 
1180377030183010304 : 24259259 203123011 758113530591776768 635750362 22240612 
779175673264902144 : 194150526 15907183 17674244 24259259 9814812 
569797178 : 24259259 56488059 20457806 26574283 1891266440 
729852258742108160 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1216770618508574721 : 56488059 20457806 24259259 939091 1349149096909668363 
1220356143941345280 : 1278694316752490500 450679540 766076479222456320 372755940 32345449 
148163990 : 24259259 56488059 20457806 26574283 1891266440 
4195195073 : 24259259 346102488 1244013067790364673 17674244 3094649957 
728663244664020992 : 24259259 56488059 20457806 26574283 1891266440 
32802933 : 33584794 15859912 281877818 61863570 

2612289973 : 24259259 56488059 20457806 26574283 1891266440 
3295594491 : 885710908013424644 429846916 2292688374 3098228941 99447959 
29095201 : 191807697 1599212660 221718444 19212550 43317553 
4212419533 : 78543084 86312103 15199327 1094360607552204801 26300616 
1325145572773302272 : 24259259 346102488 1244013067790364673 17674244 3094649957 
16062936 : 596893898 138299775 481778007 3384987100 1278694316752490500 
203291901 : 24259259 56488059 20457806 26574283 1891266440 
114800801 : 2195885540 14511951 121181001 854725669 819822691 
1170031475384733696 : 5392522 24259259 1643123766 25049056 14296273 
1185235471523287045 : 24259259 346102488 1244013067790364673 17674244 3094649957 
14815151 : 24259259 13393052 56488059 20457806 1278694316752490500 
2917985551 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1243110834450161670 : 24259259 56488059 20457806 26574283 1891266440 
1390067410028335105 : 24259259 134758540 3094649957 346102488 1244013067790364673 
91057437710

...

4098225674 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1245787984726720512 : 133081348 408793153 1267708379138535424 24259259 20457806 
1379342902443307009 : 3240396234 198118653 429846916 55060090 881974187589603328 
994081501754150915 : 24259259 346102488 3094649957 1244013067790364673 17674244 
1031560806033514497 : 24259259 346102488 1244013067790364673 17674244 3094649957 
999429250775244805 : 380648579 32828872 100248020 1656958189 6318552 
108684205 : 846411464885747712 24259259 1193633291506724864 38400130 19898168 
1088576824915812352 : 15115280 17674244 24259259 224329419 1643123766 
1296538791889506306 : 24259259 56488059 20457806 26574283 1891266440 
88993543 : 24259259 56488059 20457806 26574283 1891266440 
67728677 : 15115280 17674244 24259259 224329419 1643123766 
828342856733585413 : 24259259 346102488 1244013067790364673 17674244 3094649957 
3085368016 : 24259259 56488059 20457806 26574283 1891266440 
224088898 : 24259259 56488059 20457806 26574283 18

770721102217621504 : 24259259 346102488 1244013067790364673 17674244 3094649957 
456359293 : 24259259 346102488 1244013067790364673 17674244 3094649957 
245587515 : 1175233592592498688 783792992 2950183971 86312103 865004396681207809 
2296973810 : 15115280 17674244 24259259 224329419 1643123766 
1237080876481048576 : 53533774 1241209424 90670300 14335586 1572592322 
2174357906 : 380648579 32828872 100248020 1656958189 6318552 
28722755 : 15115280 17674244 24259259 224329419 1643123766 
1386511248666234882 : 24259259 20457806 56488059 3094649957 939091 
933638226 : 55060090 393128956 3240396234 369534928 94482117 
1094710415035154432 : 21406834 842678603305377793 2248932158 65201417 1002260754022748162 
895770960065953792 : 24259259 56488059 20457806 26574283 1891266440 
15182798 : 3240396234 16041234 15182798 1005085697018138624 607946179 
115327462 : 24259259 56488059 20457806 26574283 1891266440 
1048195050432225282 : 24259259 13393052 56488059 20457806 1278694316752490500 
818831372

565248871 : 24259259 56488059 20457806 26574283 1891266440 
2377181420 : 2195885540 14511951 121181001 854725669 819822691 
19466430 : 24259259 56488059 20457806 26574283 1891266440 
29777806 : 159955958 947605221416538112 851544830 54903435 300338214 
1063640197349298176 : 24259259 56488059 20457806 26574283 1891266440 
2897869786 : 24259259 346102488 1244013067790364673 17674244 3094649957 
737697877561081856 : 24259259 56488059 20457806 26574283 1891266440 
928760155 : 24259259 56488059 20457806 26574283 1891266440 
2784043980 : 885885503349936128 65201417 773814012068958208 15903455 1104447042993164290 
2749193982 : 24259259 56488059 20457806 26574283 1891266440 
117753607 : 24259259 56488059 20457806 26574283 1891266440 
954469582558777344 : 24259259 346102488 1244013067790364673 17674244 3094649957 
834445888860848132 : 24259259 346102488 1244013067790364673 17674244 3094649957 
546218242 : 9814812 22429979 20457806 56488059 299273962 
70274088 : 24259259 56488059 20457806 265742

2984594855 : 24259259 56488059 20457806 26574283 1891266440 
207805890 : 24259259 56488059 20457806 26574283 1891266440 
1050198991638552579 : 24259259 56488059 20457806 26574283 1891266440 
444943128 : 24259259 56488059 20457806 26574283 1891266440 
153318413 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1379127899639275520 : 24259259 346102488 1244013067790364673 17674244 3094649957 
146270233 : 133081348 408793153 1267708379138535424 24259259 20457806 
48239765 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1926381835 : 24259259 56488059 20457806 26574283 1891266440 
1389558006837391363 : 15115280 17674244 24259259 224329419 1643123766 
308023682 : 24259259 346102488 1244013067790364673 17674244 3094649957 
26868816 : 133081348 408793153 1267708379138535424 24259259 20457806 
527892987 : 3240396234 1321935792416149505 14335586 1005085697018138624 937450538446106624 
61667728 : 380648579 32828872 100248020 1656958189 6318552 
2708534521 : 24259259 30946

35348691 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1020438591896522752 : 24259259 56488059 20457806 26574283 1891266440 
1080736670 : 24259259 346102488 1244013067790364673 17674244 3094649957 
814926494573072384 : 24259259 346102488 1244013067790364673 17674244 3094649957 
780167104142118912 : 24259259 346102488 1244013067790364673 17674244 3094649957 
1348750216225296384 : 24259259 20457806 56488059 17674244 346102488 
2497878354 : 750073625915559936 32871086 1032470837780045824 119592189 15012486 
15767364 : 24259259 346102488 1244013067790364673 17674244 3094649957 
404462916 : 24259259 346102488 1244013067790364673 17674244 3094649957 
60790159 : 20457806 56488059 24259259 1385797144209096705 357606935 
1233107656539283460 : 33584794 15859912 281877818 61863570 962490153007710208 
2151411331 : 24259259 13393052 56488059 20457806 1278694316752490500 
27130071 : 24259259 346102488 1244013067790364673 17674244 3094649957 
89516920 : 24259259 13393052 56488059 2045